In [29]:
import json
import os
import csv
import urllib
from io import BytesIO
from PIL import Image

from socket import timeout
import torch.nn as nn
import torch.optim as optim

import numpy as np
import matplotlib.pyplot as plt
import torch
from torchvision import models
from torch.utils.data import Dataset, SubsetRandomSampler
from torchvision import transforms
import pickle
from plot_tool import *
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns


In [7]:
%matplotlib inline

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
device = torch.device("cuda:0") #Подрубайтунг

In [9]:
train_folder = "stats/"
points= 20

In [18]:
def saveStats(data, name):
    with open('stats/'+str(name)+".stat", 'wb') as f:
        pickle.dump(data, f)
        
def readStats(name):
    with open('stats/'+str(name), 'rb') as f:
        return pickle.load(f)

In [23]:
class CryptoDataset(Dataset):
    def __init__(self, folder, transform=None, predict_cutoff = 20):
        self.transform = transform
        self.folder = folder
        
        files = os.listdir(self.folder)
        
        self.count = len(files)
        self.games_path = list()
        self.predict_cutoff = predict_cutoff
        
        for f in os.listdir(self.folder):
            self.games_path.append(f)

    def __len__(self):
        return self.count

    def __getitem__(self, index):
        path = self.games_path[index]
        
        base_stats = readStats(path)
        
        if self.transform is not None:
            base_stats = self.transform(base_stats)

        return base_stats, base_stats, index

In [28]:
train_dataset = CryptoDataset(train_folder, transform=transforms.Compose([transforms.ToTensor(),]))
scaler = MinMaxScaler(feature_range=(-1, 1))

In [25]:
len(train_dataset)

10